In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 1

import numpy as np
from scipy.integrate import ode

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

%aimport lorenz

# The Kalman filter

The Kalman filter is a procedure to estimate the state of a system with known dynamics, but which can only be observed imperfectly.

Consider everyone's favorite example, the Lorenz attractor:

$\dot x = \sigma(y - x)$

$\dot y = x(\rho - z) - y$

$\dot z = xy - \beta z$

Given an initial condition $[x_0, y_0, z_0]$, we can use our favorite ODE solver to compute the trajectory of this initial condition.

In [2]:
q = lorenz.lorenz([5.0, 5.0, 5.0], 0.0, 10.0)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(q[:,0], q[:,1], q[:,2])

plt.show()

What if we only get to observe the system at discrete time intervals?

What if we don't even get to observe the system, just the output of some sensor?

And what if the sensor makes errors?

In [3]:
d = lorenz.observational_data

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(d[:,0], d[:,1], d[:,2], color='r', marker='.')

plt.show()

How can we use the (admittedly noisy) observations together with what we know about the underlying system?

# Some stats

There's a random variable $x$ we'd like to estimate; we already have an unbiased estimate $\hat x_0$, that is,

$$ \hat x_0 = x + \xi, $$

where $\xi \sim N(0, C)$ for some covariance matrix $C$.
Never mind where $\hat x_0$ came from for now, we'll get to that.

Now we get a measurement $y$ of $x$:

$$ y = Mx + \eta $$

where $\eta \sim N(0, Q)$.
Updated estimate $\hat x$ has the maximum a posteriori probability:

$$ \hat x = \hat x_0 + CM^*(MCM^* + Q)^{-1}(y - M\hat x_0) $$

The new estimate has

$$ \textrm{cov}(\hat x) = C - CM^*(MCM^* + Q)^{-1}MC. $$

Observe that $\textrm{cov}(\hat x) \le \textrm{cov}(\hat x_0)$.
(Derivations on the board if you want to see them.)

# Linear stochastic dynamical systems

Consider a discrete dynamical system

$$X_k = AX_{k - 1} + \xi_k.$$

$\xi_k$ is the *process noise*; $\xi_k \sim N(0, C)$.

However, we don't get to observe the process $X_k$ directly; rather, we only get to see

$$Y_k = MX_k + \eta_k,$$

where $\eta_k \sim N(0, Q)$ is the *measurement noise*.

### Prediction

Say we have an unbiased estimate $\hat x_{k-1|k-1}$ for $x_{k-1}$ based on the previous $k - 1$ observations, for which we somehow know that

$$ \textrm{cov}(\hat x_{k-1|k-1}) = P_{k|k-1}. $$

How can we update this estimate for the next timestep?

$$\hat x_{k|k-1} \equiv A\hat x_{k-1|k-1}$$

is unbiased for $x_k$.
What happens to the covariance?

$$ P_{k|k-1} \equiv \textrm{cov}(\hat x_{k|k-1}) = AP_{k-1|k-1}A^* + C$$

### Correction

Now we get some data $y_k$; how do we correct our prediction?
Well, $\hat x_{k|k-1}$ is an unbiased estimate for $x_k$, and presumably we know its covariance.
But we just did that a second ago; the a prior covariance $C$ is $P_{k|k-1}$:

$$ \hat x_{k|k} = \hat x_{k|k-1} + K_k(y_k - M\hat x_{k|k-1}) $$

where $K_k$ is the Kalman gain matrix:

$$ K_k = P_{k|k-1}M^*(MP_{k|k-1}M^* + Q)^{-1}$$

With the updated covariance:

$$ P_{k|k} = (I - K_kM)P_{k|k-1}$$

we can start over again at step $k$.

#### Appease the numerical analysts

All this can be done by keeping the Cholesky decomposition of the covariance matrix and not the inverse.

# Generalizations

We could be estimating $x_k$ based on the entire time series of measurements, including measurements that occurred before and after $k$ (Kalman smoothing, hindcasting).

There can be deterministic forcing:

$$ x_k = Ax_{k-1} + Bu_k + \xi_k $$

The underlying dynamical system can be continuous (Kalman-Bucy filter, hybrid filtering):

$$ \dot x = Ax + \xi_t $$

The underlying dynamical system need not be linear:

$$ \dot x = f(x) + \xi_t $$

Use the linearization of $f$ about the a priori estimate (extended Kalman filter).

The system can be large (think weather forecasting), in which case maintaining covariance matrices is expensive.
Then we use an *ensemble* of representative solutions to estimate the true covariance (ensemble Kalman filter).